In [ ]:
import torch
import torch.nn as nn
import time
from transformers import AutoTokenizer
from modelscope.utils.hub import snapshot_download
from detection.Loader.mymodel_file.gptJ_cloud import gptJ_cloud
from detection.Loader.mymodel_file.gptJ_edge import gptJ_edge
from detection.SVD_model import SVDED_GPTJ_EDGE_Layer

class SVD_GPTJ_Edge_Model(nn.Module):
    """包含所有SVD层的完整edge模型"""
    def __init__(self, original_edge, svd_reduce_rate, device='cpu', svd_device='cpu'):
        super().__init__()
        self.device = device
        self.svd_device = svd_device
        self.num_layers = original_edge.num_layers
        self.max_ctx = original_edge.max_ctx
        self.v_cache = [None] * self.num_layers
        
        print(f"🔄 开始SVD分解处理，压缩率: {svd_reduce_rate}")
        print(f"📊 总共需要处理 {self.num_layers} 层...")
        print(f"⚡ SVD分解设备: {svd_device}, 运行设备: {device}")
        
        # 用SVD压缩的层替换原始edge层
        self.svd_layers = nn.ModuleList()
        for i in range(self.num_layers):
            print(f"  处理第 {i+1}/{self.num_layers} 层: ", end="")
            
            # 获取原始edge层
            if(i%2):
                original_edge_layer = original_edge.layers[i]
                # 创建SVD压缩层
                svd_layer = SVDED_GPTJ_EDGE_Layer(
                    gptj_edge_layer=original_edge_layer,
                    reduce_rate=0,
                    device=device,
                    svd_device=svd_device  # 传递SVD分解设备
                )
                print("跳过压缩 (奇数层)")
                self.svd_layers.append(svd_layer)
            else:
                original_edge_layer = original_edge.layers[i]
                # 创建SVD压缩层
                print(f"正在进行SVD分解 (压缩率: {svd_reduce_rate})...")
                
                # 计算SVD分解的时间
                svd_start_time = time.time()
                svd_layer = SVDED_GPTJ_EDGE_Layer(
                    gptj_edge_layer=original_edge_layer,
                    reduce_rate=svd_reduce_rate,
                    device=device,
                    svd_device=svd_device  # 传递SVD分解设备
                )
                svd_end_time = time.time()
                print(f"    ✅ 第 {i+1} 层SVD分解完成 (耗时: {svd_end_time - svd_start_time:.2f}秒)")
                self.svd_layers.append(svd_layer)
        
        print(f"🎉 所有层的SVD分解处理完成！")
    
    def forward_all_layers(self, x, attn_weights_list):
        """
        处理所有层的前向传播
        x: Tensor [batch_size, seq_len, hidden]
        attn_weights_list: List[Tensor] - 每层的注意力权重
        返回: 最终的x输出
        """
        current_x = x
        for layer_idx in range(self.num_layers):
            attn_weights = attn_weights_list[layer_idx]
            # 使用SVD压缩的层
            self.v_cache[layer_idx], current_x = self.svd_layers[layer_idx].forward_cache(
                current_x, self.v_cache[layer_idx], attn_weights
            )
            
            # 应用sliding window到缓存
            if self.v_cache[layer_idx] is not None and self.v_cache[layer_idx].size(1) > self.max_ctx:
                self.v_cache[layer_idx] = self.v_cache[layer_idx][:, -self.max_ctx:, :]
        
        return current_x

class GPTJPipeline:
    def __init__(self, model_name='AI-ModelScope/gpt-j-6b', device_cloud='cuda:0', device_edge='cpu', svd_reduce_rate=0.5, use_compile=True):
        print(f"🚀 初始化GPTJPipeline...")
        print(f"📋 配置信息:")
        print(f"   - 模型: {model_name}")
        print(f"   - 云端设备: {device_cloud}")
        print(f"   - 边缘设备: {device_edge}")
        print(f"   - SVD压缩率: {svd_reduce_rate}")
        
        # 使用 ModelScope 下载模型
        print(f"📥 使用ModelScope下载模型 {model_name}...")
        model_dir = snapshot_download(
            repo_id=model_name,
            cache_dir='./gpt-j-6b'
        )
        print(f"✅ 模型下载完成，路径: {model_dir}")
        
        # 使用本地模型路径加载 tokenizer
        print(f"🔤 加载tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
        
        # 设置 pad_token 为 eos_token（GPT-J 没有 pad_token）
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        print(f"☁️  加载云端模型到 {device_cloud}...")
        self.cloud       = gptJ_cloud(model_name=model_dir).to(device_cloud)
        print(f"🖥️  加载边缘模型到CPU...")
        # 强制 edge 放在 CPU
        original_edge    = gptJ_edge (model_name=model_dir).to('cpu')
        self.embed       = self.cloud.model.transformer.wte
        self.ln_f        = self.cloud.model.transformer.ln_f
        self.lm_head     = self.cloud.model.lm_head
        self.num_layers  = len(self.cloud.q_weights)
        
        print(f"📊 模型共有 {self.num_layers} 层")
        
        # SVD压缩参数
        self.svd_reduce_rate = svd_reduce_rate
        self.use_compile = use_compile
        
        # 创建整个SVD edge模型
        print(f"🔧 创建SVD边缘模型...")
        # 如果有GPU，先在GPU上进行SVD分解，然后移到CPU
        svd_device = device_cloud if torch.cuda.is_available() else 'cpu'
        print(f"🔧 SVD分解将在 {svd_device} 上进行...")
        
        self.svd_edge_model = SVD_GPTJ_Edge_Model(
            original_edge=original_edge,
            svd_reduce_rate=svd_reduce_rate,
            device='cpu',  # 最终运行在CPU上
            svd_device=svd_device  # 但SVD分解在GPU上进行
        )
        
        print(f"✅ GPTJPipeline初始化完成！")
        print(f"🎯 准备开始推理，SVD压缩率: {self.svd_reduce_rate}")
    
    def generate(self, prompt, max_length=50, temperature=1.0, top_k=50):
        print(f"🔄 开始文本生成...")
        print(f"📝 输入提示: '{prompt}'")
        print(f"⚙️  生成参数: max_length={max_length}, temperature={temperature}, top_k={top_k}")
        
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt')[0].tolist()
        outputs   = input_ids.copy()

        # reset caches for a fresh generation
        print(f"🗂️  清空缓存...")
        for i in range(self.num_layers):
            self.cloud.k_cache[i] = None
            self.svd_edge_model.v_cache[i] = None

        # 统计变量
        cloud_time = 0.0
        edge_time  = 0.0
        layer_calls= 0
        net_time=0.0
        bandwidth=10 #MB/s

        # 上下文窗口大小
        max_ctx = self.cloud.max_ctx

        print(f"🔥 预热阶段：处理 {len(input_ids)} 个提示token...")
        # 预热缓存：将 prompt 中每个 token 走一次 forward_cache
        for pos, token_id in enumerate(input_ids):
            print(f"  处理提示token {pos+1}/{len(input_ids)}")
            # clamp 位置，防止越界
            pos_clamped = pos if pos < max_ctx else max_ctx - 1
            cur_id = torch.tensor([[token_id]]).to(self.embed.weight.device)
            
            # GPT-J 没有位置embedding，直接使用 token embedding
            x = self.embed(cur_id)
            
            # 收集所有层的注意力权重
            attn_weights_list = []
            for layer_idx in range(self.num_layers):
                # cloud on GPU
                if hasattr(torch.cuda, 'synchronize'):
                    torch.cuda.synchronize()
                t0 = time.time()
                _, _, attn_weights = self.cloud.forward_cache(x, layer_idx)
                if hasattr(torch.cuda, 'synchronize'):
                    torch.cuda.synchronize()
                cloud_time += time.time() - t0
                
                # 移动到CPU并收集注意力权重
                attn_weights_cpu = attn_weights.to('cpu')
                attn_weights_list.append(attn_weights_cpu)
                
                # 计算网络传输时间
                elements = attn_weights_cpu.numel() * attn_weights_cpu.element_size()  # B
                net_time += elements / bandwidth / 1024 / 1024  # s
                
                layer_calls += 1
            
            # 使用SVD edge模型处理所有层
            t1 = time.time()
            x_cpu = x.to('cpu')
            final_x = self.svd_edge_model.forward_all_layers(x_cpu, attn_weights_list)
            edge_time += time.time() - t1
            
            # 回到GPU为下一个token做准备
            x = final_x.to(self.embed.weight.device)
            elements = x.numel() * x.element_size()  # B
            net_time += elements / bandwidth / 1024 / 1024

        print(f"🎯 生成阶段：开始生成新token...")
        # 真实生成阶段
        for token_idx in range(max_length):
            if token_idx % 5 == 0:  # 每5个token显示一次进度
                print(f"  生成进度: {token_idx}/{max_length}")
                
            cur_id = torch.tensor([[outputs[-1]]]).to(self.embed.weight.device)
            x = self.embed(cur_id)
            
            # 收集所有层的注意力权重
            attn_weights_list = []
            for layer_idx in range(self.num_layers):
                # use cache-enabled forward so attention spans all previous tokens
                if hasattr(torch.cuda, 'synchronize'):
                    torch.cuda.synchronize()
                t0 = time.time()
                _, _, attn_weights = self.cloud.forward_cache(x, layer_idx)
                if hasattr(torch.cuda, 'synchronize'):
                    torch.cuda.synchronize()
                cloud_time += time.time() - t0

                # 移动到CPU并收集注意力权重
                attn_weights_cpu = attn_weights.to('cpu')
                attn_weights_list.append(attn_weights_cpu)
                
                # 计算网络传输时间
                elements = attn_weights_cpu.numel() * attn_weights_cpu.element_size()  # B
                net_time += elements / bandwidth / 1024 / 1024
                
                layer_calls += 1
            
            # 使用SVD edge模型处理所有层
            t1 = time.time()
            x_cpu = x.to('cpu')
            final_x = self.svd_edge_model.forward_all_layers(x_cpu, attn_weights_list)
            edge_time += time.time() - t1
            
            # 回到GPU继续
            x = final_x.to(self.embed.weight.device)
            elements = x.numel() * x.element_size()  # B
            net_time += elements / bandwidth / 1024 / 1024
            
            # final normalization and LM head to get logits
            x = self.ln_f(x)
            logits = self.lm_head(x)
            
            # 用 top-k + 温度采样代替贪心 argmax
            next_logits = logits[:, -1, :] / temperature
            topk_vals, topk_idx = torch.topk(next_logits, k=top_k, dim=-1)
            probs = torch.softmax(topk_vals, dim=-1)
            next_id = topk_idx[0, torch.multinomial(probs, num_samples=1).item()].item()
            outputs.append(next_id)
            
            if next_id == self.tokenizer.eos_token_id:
                print(f"  遇到结束符，提前结束生成")
                break

        print(f"📊 生成完成，统计性能数据...")
        # 打印平均耗时
        if layer_calls > 0:
            avg_cloud_time = cloud_time / (layer_calls / self.num_layers)
            avg_edge_time = edge_time / (layer_calls / self.num_layers)
            avg_net_time = net_time / (layer_calls / self.num_layers)
            print(f"☁️  平均GPU(cloud)每token耗时: {avg_cloud_time:.4f}s")
            print(f"🖥️  平均CPU(edge)每token耗时: {avg_edge_time:.4f}s")
            print(f"🌐 平均网络传输每token耗时: {avg_net_time:.4f}s")
            print(f"🔄 总平均每token耗时: {avg_cloud_time + avg_edge_time + avg_net_time:.4f}s")
            
        return self.tokenizer.decode(outputs, clean_up_tokenization_spaces=True)



In [ ]:

model_name = 'AI-ModelScope/gpt-j-6b'
device_cloud = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device_edge = 'cpu'

# 检查CUDA可用性
if torch.cuda.is_available():
    print(f"🎮 检测到CUDA设备，将使用GPU进行云端计算")
    print(f"🔧 GPU设备: {torch.cuda.get_device_name(0)}")
else:
    print(f"⚠️  未检测到CUDA设备，将使用CPU进行云端计算")

# 测试不同的SVD压缩率
svd_rate=0


print(f"\n{'='*60}")
print(f"🧪 测试SVD压缩率: {svd_rate}")
print(f"{'='*60}")

try:
    pipeline = GPTJPipeline(
        model_name=model_name, 
        device_cloud=device_cloud, 
        device_edge=device_edge,
        svd_reduce_rate=svd_rate
    )
    

    prompt = "Once upon a time"
    print(f"\n💬 提示词: '{prompt}'")
    
    print(f"⏱️  开始生成文本...")
    start_time = time.time()
    generated_text = pipeline.generate(prompt, max_length=20)
    end_time = time.time()
    
    print(f"\n📝 生成的文本:")
    print(f"   {generated_text}")
    print(f"⏱️  总生成时间: {end_time - start_time:.2f}秒")
    
except Exception as e:
    print(f"❌ 测试失败: {str(e)}")
    import traceback
    traceback.print_exc()

print(f"{'='*60}")
print(f"🏁 SVD压缩率 {svd_rate} 测试完成")
print(f"{'='*60}")

print(f"\n🎉 所有测试完成！")